## Mithilfe von PyMuPDF pdf-Dateien auslesen: SPD


In [1]:
! pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 46.4 MB/s eta 0:00:00


In [2]:
## pdf-Datei anschauen: Die ersten 6 Seiten enthalten Deckblatt + Inhaltsverzeichnis - das kann also übersprungen werden

# Als erstes die Formatinformation extrahieren: Also Text mit Schriftgröße und Dicke der Schrift, um später die Überschriften zu finden und
# zur Gliederung verwenden zu können

# Text ab vierter Seite bis vorletzte Seite extrahieren inkl. Formatinfos:

import fitz

def extract_text_with_formatting(pdf_path, skip_pages=3):
    doc = fitz.open(pdf_path)
    text_data = []

    for page_num in range(skip_pages, 66):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]  # Extrahiert Textblöcke

        for block in blocks:
            if block["type"] == 0:  # Textblock
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Speichere Text + Schriftgröße + Fettdruck-Info
                        text_data.append({
                            "text": span["text"],
                            "size": span["size"],  # Schriftgröße
                            "bold": span["flags"] & 2 != 0  # Prüft, ob der Text fett ist
                        })

    return text_data

# PDF-Dateipfad
pdf_path = "/content/Wahlprogramm_SPD.pdf"

# Text mit Formatierungsinformationen extrahieren
text_data = extract_text_with_formatting(pdf_path)

# Zeige die ersten paar Textabschnitte mit Formatierung
for item in text_data[:1000]:
    print(f"Text: {item['text']}, Size: {item['size']}, Bold: {item['bold']}")


Text: 4, Size: 7.0, Bold: False
Text: Mehr für Dich. Besser für Deutschland., Size: 7.0, Bold: False
Text: Regierungsprogramm der SPD für die Bundestagswahl 2025, Size: 7.0, Bold: False
Text: Worauf es jetzt ankommt, Size: 16.0, Bold: False
Text: Am 23. Februar 2025 findet die vorgezogene Bundestagswahl statt. Es geht um viel bei dieser Wahl. , Size: 11.0, Bold: False
Text: Wir leben in schwierigen Zeiten: Es herrscht Krieg mitten in Europa, unsere Unternehmen stehen in , Size: 11.0, Bold: False
Text: einem verschärften internationalen Wettbewerb, der Zusammenhalt im Land steht auf der Probe., Size: 11.0, Bold: False
Text: Deutschland steht vor bedeutenden Entscheidungen:, Size: 11.0, Bold: False
Text: • , Size: 10.0, Bold: False
Text: Investieren wir , Size: 11.0, Bold: False
Text: jetzt, Size: 11.0, Bold: True
Text:  weiter kraftvoll in unser Land? Oder lassen wir unsere Bahn, unsere Straßen , Size: 11.0, Bold: False
Text: und Brücken verkommen?, Size: 11.0, Bold: False
Text: • , Siz

In [3]:
## Output anschauen: Normale Schrift ist in Schriftgröße 11, Überschriften sind größer, Seitenzahlen stehen in einzelnen Zeilen
## Kopfzeile in Schriftgröße 7 (zweizeilig)


import pandas as pd
import re


def split_sections(text_data, min_size=11):
    sections = []
    current_section = []

    for item in text_data:
        text = item["text"]

        if item["size"] > min_size:  # Überschriften überspringen + Abschnitte definieren
            if current_section:
                sections.append("".join(current_section).strip())
            current_section = []

        elif item["size"] < 11: # Nicht die Kopfzeile anhängen
            continue

        elif text == "":  # Leere Zeilen überspringen
            continue

        elif text.isupper():  # Kopfzeilenschriftzug überspringen
            continue

        else:
            current_section.append(text)

    if current_section:  # Letzten Abschnitt hinzufügen
        sections.append("".join(current_section).strip())

    return sections

# Text mit Formatierungsinformationen extrahieren
text_data = extract_text_with_formatting(pdf_path)

# Abschnitte extrahieren
sections = split_sections(text_data)

# Tabelle speichern
df = pd.DataFrame(sections, columns=["Abschnitt"])
print(df[:10])


                                           Abschnitt
0  Am 23. Februar 2025 findet die vorgezogene Bun...
1  Viele Menschen machen sich derzeit Sorgen um d...
2  Dies ist das erste, aber sicher nicht das letz...
3  Made in Germany – was war und ist das für ein ...
4  Eines können wir versprechen: Wir als SPD, wir...
5  Gute Bildung für alle Kinder und Jugendlichen ...
6  Darüber reden, was alles nicht gut ist, das kö...
7  Viel ist in diesen Zeiten in der Politik, der ...
8  Wohnen ist ein Menschenrecht! Wer wohnt, hat e...
9  Wer jeden Tag arbeitet, muss ein selbstbestimm...


In [6]:
# Speichern der Tabelle als CSV-Datei
df.to_csv("/content/spd.csv", index=False, encoding="utf-8")